# 🚀 术语提取 (Terminology Extraction)

**代码融合了三种核心算法来挖掘高质量术语：**
1.  **spaCy (Linguistic):** 利用句法依赖分析提取名词短语 (Noun Chunks)。
2.  **NLTK (Statistical):** 利用互信息 (PMI) 提取高频双词搭配 (Bigram Collocations)。
3.  **TF-IDF (Weighting):** 计算术语在文档中的独特性权重，过滤通用词汇。

---

In [10]:
# === Cell 1: 环境检查与依赖安装 ===
import sys
import os
import pandas

# 1. 取消行数限制（显示所有行）
pd.set_option('display.max_rows', None)  
# 2. 取消列数限制（显示所有列）
pd.set_option('display.max_columns', None)  
# 3. 调整列宽（避免列内容被截断，设置为100字符宽度）
pd.set_option('display.max_colwidth', 100)  
# 4. 禁止列名换行（保持表格整洁）
pd.set_option('display.expand_frame_repr', False)

# 定义增强版依赖库
required_packages = [
    "pymupdf",       # PDF 解析
    "pytesseract",   # OCR
    "spacy",         # 语法分析
    "pandas",        # 数据处理
    "openpyxl",      # Excel 导出
    "requests",      # 网络请求
    "pillow",        # 图片处理
    "nltk",          # 自然语言处理 (搭配词提取)
    "scikit-learn"   # 机器学习 (TF-IDF 计算)
]

print("🛠️ 正在检查并安装缺失库...")
for package in required_packages:
    try:
        __import__(package)
    except ImportError:
        print(f"  ⬇️ 正在安装: {package}...")
        !{sys.executable} -m pip install {package} -q

# 下载 NLP 模型数据
import spacy
import nltk

try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    print("  ⬇️ 下载 spaCy 模型...")
    !{sys.executable} -m spacy download en_core_web_sm
    nlp = spacy.load("en_core_web_sm")

print("  ⬇️ 检查 NLTK 数据包...")
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)

print("✅ 所有环境配置就绪！")

🛠️ 正在检查并安装缺失库...
  ⬇️ 正在安装: pillow...
  ⬇️ 正在安装: scikit-learn...
  ⬇️ 检查 NLTK 数据包...
✅ 所有环境配置就绪！


In [11]:
# === Cell 2: 智能 PDF 解析器 (支持分页处理) ===
import fitz  # PyMuPDF
import pytesseract
from PIL import Image
import io
import requests

# 设置 Tesseract 路径 (Windows用户需根据实际情况修改)
# 如果在 Mac/Linux 上通常不需要这行，或者路径不同
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
if not os.path.exists(pytesseract.pytesseract.tesseract_cmd):
    # 备用路径尝试
    pytesseract.pytesseract.tesseract_cmd = r'd:\Tesseract-OCR\tesseract.exe'

def download_sample_pdf(url, filename="sample_paper.pdf"):
    if os.path.exists(filename):
        print(f"📂 使用本地文件: {filename}")
        return filename
    print(f"⬇️ 正在下载: {url}...")
    try:
        response = requests.get(url, stream=True)
        with open(filename, 'wb') as f:
            f.write(response.content)
        return filename
    except Exception as e:
        print(f"❌ 下载失败: {e}")
        return None

def parse_pdf_by_pages(pdf_path):
    """
    解析 PDF 并返回一个列表，列表每项代表一页的文本。
    这对于 TF-IDF 很重要，因为我们需要区分文档的不同部分。
    """
    if not pdf_path: return []
    
    doc = fitz.open(pdf_path)
    pages_content = []
    print(f"🚀 开始解析 PDF (共 {len(doc)} 页)...")

    for i, page in enumerate(doc):
        text = page.get_text()
        
        # 简单的扫描件检测
        if len(text.strip()) < 50:
            print(f"  ⚠️ 第 {i+1} 页疑似扫描件，尝试 OCR...")
            try:
                pix = page.get_pixmap(matrix=fitz.Matrix(2, 2))
                img = Image.open(io.BytesIO(pix.tobytes("png")))
                text = pytesseract.image_to_string(img, lang='eng')
            except:
                text = "" # OCR 失败忽略
        
        # 清洗基础格式
        text = text.replace('\n', ' ').strip()
        pages_content.append(text)
        
    return pages_content

# 准备数据
pdf_url = "https://arxiv.org/pdf/1706.03762.pdf" # Attention Is All You Need
local_pdf = download_sample_pdf(pdf_url, "paper.pdf")
raw_pages = parse_pdf_by_pages(local_pdf)
print(f"✅ 解析完成，获取 {len(raw_pages)} 页内容。")

📂 使用本地文件: paper.pdf
🚀 开始解析 PDF (共 25 页)...
  ⚠️ 第 19 页疑似扫描件，尝试 OCR...
  ⚠️ 第 20 页疑似扫描件，尝试 OCR...
  ⚠️ 第 21 页疑似扫描件，尝试 OCR...
  ⚠️ 第 22 页疑似扫描件，尝试 OCR...
  ⚠️ 第 23 页疑似扫描件，尝试 OCR...
  ⚠️ 第 24 页疑似扫描件，尝试 OCR...
  ⚠️ 第 25 页疑似扫描件，尝试 OCR...
✅ 解析完成，获取 25 页内容。


In [12]:
# === Cell 3: 核心算法融合 (TF-IDF + spaCy + NLTK) ===
import pandas as pd
import re
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.collocations import BigramCollocationFinder, BigramAssocMeasures
from nltk.corpus import stopwords

class TerminologyExtractor:
    def __init__(self, nlp_model):
        self.nlp = nlp_model
        self.stop_words = set(stopwords.words('english'))
        # 添加一些学术论文特有的无效词
        self.stop_words.update(['et', 'al', 'figure', 'table', 'section', 'fig', 'eq', 'p.', 'pp.', 'doi'])

    def preprocess(self, text):
        # 移除引用 [1], 标点等
        text = re.sub(r'\[\d+\]', '', text)
        text = re.sub(r'[^a-zA-Z\s-]', '', text) # 仅保留字母和连字符
        return text

    def get_spacy_candidates(self, text):
        """使用 spaCy 提取名词短语 (Noun Chunks)"""
        doc = self.nlp(text[:1000000]) # 限制长度防止内存溢出
        candidates = []
        for chunk in doc.noun_chunks:
            term = chunk.text.lower().strip()
            # 过滤规则：长度>2，不含数字，不在停用词表中
            if len(term) > 2 and term not in self.stop_words:
                # 进阶过滤：只保留包含空格的词组 (single word 往往太通用)
                if " " in term:
                    candidates.append(term)
        return candidates

    def get_nltk_collocations(self, text, top_n=100):
        """使用 NLTK 提取高频搭配 (Bigrams)"""
        tokens = nltk.word_tokenize(text.lower())
        # 仅保留纯字母词
        tokens = [w for w in tokens if w.isalpha() and w not in self.stop_words]
        
        bigram_measures = BigramAssocMeasures()
        finder = BigramCollocationFinder.from_words(tokens)
        # 过滤掉出现少于 3 次的搭配
        finder.apply_freq_filter(3)
        
        # 返回前 Top N 的搭配词 (按频率)
        return [' '.join(bigram) for bigram in finder.nbest(bigram_measures.raw_freq, top_n)]

    def calculate_tfidf_scores(self, pages_list, candidates):
        """计算候选词的 TF-IDF 分数"""
        # 仅针对我们要分析的候选词建立词汇表
        vocab = list(set(candidates))
        if not vocab: return {}
        
        # TF-IDF 设置：ngram范围1-3，使用自定义词汇表
        vectorizer = TfidfVectorizer(
            vocabulary=vocab, 
            ngram_range=(1, 3), 
            stop_words='english',
            norm='l2'
        )
        
        try:
            tfidf_matrix = vectorizer.fit_transform(pages_list)
            # 对所有页面的分数求和或取平均，代表该词在全文的重要性
            feature_names = vectorizer.get_feature_names_out()
            # Sum along columns (pages)
            scores = tfidf_matrix.sum(axis=0).A1 
            return dict(zip(feature_names, scores))
        except ValueError:
            return {}

    def find_context(self, term, full_text):
        """查找例句"""
        start = full_text.find(term)
        if start != -1:
            # 截取前后 50 个字符
            s = max(0, start - 50)
            e = min(len(full_text), start + len(term) + 50)
            return "..." + full_text[s:e].replace('\n', ' ') + "..."
        return ""

    def process(self, pages_content):
        full_text = " ".join(pages_content)
        clean_text = self.preprocess(full_text)
        
        print("1️⃣ 正在运行 spaCy 句法提取...")
        spacy_terms = self.get_spacy_candidates(clean_text)
        
        print("2️⃣ 正在运行 NLTK 搭配分析...")
        nltk_terms = self.get_nltk_collocations(clean_text)
        
        # 合并候选集 (去重)
        all_candidates = list(set(spacy_terms + nltk_terms))
        print(f"   - 提取到 {len(all_candidates)} 个唯一候选术语")
        
        print("3️⃣ 正在计算 TF-IDF 权重...")
        tfidf_scores = self.calculate_tfidf_scores(pages_content, all_candidates)
        
        # 统计原始频率
        freq_counts = Counter(spacy_terms + nltk_terms) # 这里简化统计，实际应在原问中统计
        # 更精准的频率统计：在原清洗文本中统计
        final_stats = []
        
        print("4️⃣ 正在整合与评分...")
        for term in all_candidates:
            count = full_text.lower().count(term)
            if count < 2: continue # 过滤低频孤例
            
            tfidf = tfidf_scores.get(term, 0)
            
            # === 评分公式 ===
            # 综合分 = Log(频率) * TF-IDF + 来源加权
            # 如果是 spaCy (语法正确) 且是 NLTK (统计显著)，加分
            is_spacy = 1 if term in spacy_terms else 0
            is_nltk = 1 if term in nltk_terms else 0
            
            composite_score = (tfidf * 10) * (1 + (is_spacy * 0.2) + (is_nltk * 0.2))
            
            final_stats.append({
                "Term": term,
                "Frequency": count,
                "TF-IDF Score": round(tfidf, 4),
                "Composite Score": round(composite_score, 4),
                "Source": "Both" if is_spacy and is_nltk else ("spaCy" if is_spacy else "NLTK"),
                "Context": self.find_context(term, full_text)
            })
            
        df = pd.DataFrame(final_stats)
        return df.sort_values(by="Composite Score", ascending=False)

# 实例化并运行
extractor = TerminologyExtractor(nlp)
df_final = extractor.process(raw_pages)

# 显示前 150 个高质量术语
display(df_final.head(150))

1️⃣ 正在运行 spaCy 句法提取...
2️⃣ 正在运行 NLTK 搭配分析...
   - 提取到 1333 个唯一候选术语
3️⃣ 正在计算 TF-IDF 权重...
4️⃣ 正在整合与评分...


,Term,Frequency,TF-IDF Score,Composite Score,Source,Context
165,opioid misuse,50,2.4244,33.9409,Both,...10-z A videogame for perceived risk of harm from opioid misuse in adolescents: a randomized ...
71,perceived risk,33,1.6959,23.7420,Both,...oi.org/10.1038/s44360-025-00010-z A videogame for perceived risk of harm from opioid misuse ...
65,nature health,21,1.3833,19.3667,Both,...ature Health | Volume 1 | January 2026 | 78–89 78 nature health Article https://doi.org/10.10...
59,great risk,29,1.3237,18.5316,Both,...art versus 23% of control participants reported ‘great risk’ with no statistically significa...
58,et al,28,1.4672,17.6064,spaCy,"... Of the 1,062 unique indi- viduals screened, 839 met all inclusion criteria and 533 enroled: ..."
163,extended data,12,1.3587,16.3047,NLTK,
178,extended data table,12,1.3587,16.3047,spaCy,
17,health article,7,1.2976,15.5710,NLTK,
141,randomization group,15,1.0771,15.0789,Both,"...sing a linear mixed model with fixed effects for randomization group, time, interaction of r..."
174,substance use,17,0.9506,13.3086,Both,... no prior opioid misuse and ‘high-risk’ based on substance use or mental health screens—agre...


In [8]:
# === Cell 4: 结果美化与导出 ===

# 过滤掉太短的术语或纯数字噪音
filtered_df = df_final[df_final['Term'].str.len() > 4].copy()

# 重命名列以便阅读
export_columns = {
    "Term": "术语 (English)",
    "Frequency": "出现频次",
    "Composite Score": "相关性评分",
    "Source": "提取来源",
    "Context": "原文例句"
}
export_df = filtered_df.rename(columns=export_columns)

# 添加一列空白的中文翻译列
export_df.insert(1, "中文翻译", "")

filename = "Smart_Glossary_TFIDF.xlsx"
try:
    export_df.to_excel(filename, index=False)
    print(f"🎉 处理完毕！高质量术语表已保存至: {filename}")
    print(f"📊 共提取术语: {len(export_df)} 个")
except Exception as e:
    print(f"❌ 保存失败: {e}")

🎉 处理完毕！高质量术语表已保存至: Smart_Glossary_TFIDF.xlsx
📊 共提取术语: 212 个
